### Importing pyspark capabilities and session

In [1]:
from pyspark.sql import SparkSession
from pyspark import SparkContext
from pyspark.sql.functions import *
from pyspark.sql.types import *
from pyspark.sql.functions import when, col, sum, desc
import pyspark.sql
from datetime import datetime
from pyspark.sql.functions import udf
from pyspark.sql.types import DateType
from pyspark.sql.functions import year, month
from pyspark.sql.types import IntegerType,StringType


VBox()

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
0,application_1670463192599_0001,pyspark,idle,Link,Link,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [3]:
#Initiating a spark session 
spark = SparkSession \
    .builder \
    .appName("aws") \
    .getOrCreate()
    
sc = SparkContext.getOrCreate()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [ ]:
# Importing csv files from S3
y19 = spark.read.csv("s3://airlinebdai/y19.csv",inferSchema = "true", header = "true")
y20 = spark.read.csv("s3://airlinebdai/y20.csv",inferSchema = "true", header = "true")
y21 = spark.read.csv("s3://airlinebdai/y21.csv", inferSchema = "true", header = "true")
covid = spark.read.csv("s3://airlinebdai/United_States_COVID-19_Cases_and_Deaths_by_State_over_Time_-_ARCHIVED.csv",  inferSchema = "true", header = "true")



In [62]:
#Shape of 2019 data
print((y19.count(), len(y19.columns)))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

(8091684, 43)

In [63]:
#Shape of 2020 data
print((y20.count(), len(y20.columns)))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

(5022397, 43)

In [64]:
#Shape of 2021 data
print((y21.count(), len(y21.columns)))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

(6311871, 43)

In [65]:
#Shape of covid data
print((covid.count(), len(covid.columns)))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

(60060, 3)

In [7]:
#Changing boolean to numerical columns
y19 = y19.withColumn('Cancelled', when(col('Cancelled') == True, 1).otherwise(0))
y20 = y20.withColumn('Cancelled', when(col('Cancelled') == True, 1).otherwise(0))
y21 = y21.withColumn('Cancelled', when(col('Cancelled') == True, 1).otherwise(0))



y19 = y19.withColumn('Diverted', when(col('Diverted') == True, 1).otherwise(0))
y20 = y20.withColumn('Diverted', when(col('Diverted') == True, 1).otherwise(0))
y21 = y21.withColumn('Diverted', when(col('Diverted') == True, 1).otherwise(0))



VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [9]:
y19.show(5)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-------------------+---------+------+----+---------+--------+----------+-------+---------------+--------+-------+---------------+-------+--------------+-----------------+--------+----+-------+-----+----------+---------+---------------------------+-------------------------------+-----------------+-------------------------------+---------------+------------------+---------------+-----------+---------------+-------------+----------------+----------------+------------+---------+-------------+--------+--------------------+----------+--------+--------+------------------+
|         FlightDate|  Airline|Origin|Dest|Cancelled|Diverted|CRSDepTime|DepTime|DepDelayMinutes|DepDelay|ArrTime|ArrDelayMinutes|AirTime|CRSElapsedTime|ActualElapsedTime|Distance|Year|Quarter|Month|DayofMonth|DayOfWeek|IATA_Code_Marketing_Airline|Flight_Number_Marketing_Airline|Operating_Airline|Flight_Number_Operating_Airline|OriginAirportID|OriginAirportSeqID| OriginCityName|OriginState|OriginStateName|DestAirportID|Dest

In [57]:
#Combining all years data 
merged = y19.union(y20)
merged = merged.union(y21)
print((merged.count(), len(merged.columns)))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

(19425952, 43)

In [79]:
merged.show(5)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-------------------+--------+---------+--------+---------+
|         FlightDate|DepDelay|  Airline|ArrDelay|Cancelled|
+-------------------+--------+---------+--------+---------+
|2019-04-01 00:00:00|    -3.0|Envoy Air|   -15.0|        0|
|2019-04-02 00:00:00|   -12.0|Envoy Air|   -17.0|        0|
|2019-04-03 00:00:00|    -9.0|Envoy Air|   -23.0|        0|
|2019-04-04 00:00:00|   143.0|Envoy Air|   136.0|        0|
|2019-04-05 00:00:00|     4.0|Envoy Air|     5.0|        0|
+-------------------+--------+---------+--------+---------+
only showing top 5 rows


### Analyzing the Airlines were affected the most pre and post pandemic

In [11]:
#For year 2019
a = y19.groupBy("Airline").sum("Cancelled").withColumnRenamed("sum(Cancelled)", "Cancelled_Count") 
a.sort(a.Cancelled_Count.desc()).show(truncate=False)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-----------------------------------------+---------------+
|Airline                                  |Cancelled_Count|
+-----------------------------------------+---------------+
|Southwest Airlines Co.                   |33622          |
|American Airlines Inc.                   |20151          |
|SkyWest Airlines Inc.                    |17349          |
|Envoy Air                                |11405          |
|Republic Airlines                        |7679           |
|Mesa Airlines Inc.                       |6597           |
|Comair Inc.                              |6494           |
|ExpressJet Airlines Inc.                 |5650           |
|United Air Lines Inc.                    |5384           |
|Capital Cargo International              |4680           |
|Endeavor Air Inc.                        |4236           |
|JetBlue Airways                          |3764           |
|Air Wisconsin Airlines Corp              |3762           |
|Spirit Air Lines                       

In [25]:
#For 2020
b = y20.groupBy("Airline").sum("Cancelled").withColumnRenamed("sum(Cancelled)", "Cancelled_Count") 
b.sort(b.Cancelled_Count.desc()).show(truncate=False)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+----------------------------------------+---------------+
|Airline                                 |Cancelled_Count|
+----------------------------------------+---------------+
|Southwest Airlines Co.                  |77842          |
|American Airlines Inc.                  |34262          |
|Delta Air Lines Inc.                    |28769          |
|SkyWest Airlines Inc.                   |24881          |
|United Air Lines Inc.                   |22593          |
|Allegiant Air                           |14296          |
|Comair Inc.                             |14099          |
|Envoy Air                               |11194          |
|Republic Airlines                       |10516          |
|JetBlue Airways                         |7982           |
|Mesa Airlines Inc.                      |7974           |
|Endeavor Air Inc.                       |6940           |
|Capital Cargo International             |5505           |
|Alaska Airlines Inc.                    |5481          

In [26]:
#For 2021
c = y21.groupBy("Airline").sum("Cancelled").withColumnRenamed("sum(Cancelled)", "Cancelled_Count") 
c.sort(c.Cancelled_Count.desc()).show(truncate=False)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-----------------------------------------+---------------+
|Airline                                  |Cancelled_Count|
+-----------------------------------------+---------------+
|Southwest Airlines Co.                   |23505          |
|American Airlines Inc.                   |16478          |
|SkyWest Airlines Inc.                    |13323          |
|United Air Lines Inc.                    |6107           |
|Envoy Air                                |5686           |
|Spirit Air Lines                         |5661           |
|Republic Airlines                        |5647           |
|Mesa Airlines Inc.                       |4763           |
|Allegiant Air                            |3829           |
|Delta Air Lines Inc.                     |3631           |
|JetBlue Airways                          |3299           |
|Alaska Airlines Inc.                     |3217           |
|Commutair Aka Champlain Enterprises, Inc.|2489           |
|Air Wisconsin Airlines Corp            

### Analysing state wise cancellations before and during the pandemic

In [31]:

d = y19.groupBy("DestStateName").sum("Cancelled").withColumnRenamed("sum(Cancelled)", "Cancelled_Count") 
d.sort(d.Cancelled_Count.desc()).show(truncate=False)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------+---------------+
|DestStateName |Cancelled_Count|
+--------------+---------------+
|Texas         |18607          |
|Illinois      |17536          |
|California    |14911          |
|New York      |10276          |
|Florida       |7981           |
|Colorado      |7316           |
|Virginia      |7031           |
|North Carolina|6972           |
|Pennsylvania  |5804           |
|New Jersey    |5168           |
|Missouri      |3306           |
|Michigan      |3304           |
|Massachusetts |3260           |
|Arizona       |3018           |
|Washington    |3000           |
|Georgia       |2937           |
|Ohio          |2784           |
|Maryland      |2694           |
|Tennessee     |2487           |
|Nevada        |2423           |
+--------------+---------------+
only showing top 20 rows

In [33]:
e = y20.groupBy("DestStateName").sum("Cancelled").withColumnRenamed("sum(Cancelled)", "Cancelled_Count") 
e.sort(e.Cancelled_Count.desc()).show(truncate=False)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------+---------------+
|DestStateName |Cancelled_Count|
+--------------+---------------+
|Texas         |32786          |
|California    |30523          |
|Florida       |25562          |
|Illinois      |20942          |
|North Carolina|15935          |
|New York      |14147          |
|Colorado      |13453          |
|Georgia       |13128          |
|Virginia      |11319          |
|Nevada        |10247          |
|Pennsylvania  |9217           |
|Arizona       |8479           |
|Washington    |6681           |
|Missouri      |6346           |
|Tennessee     |6321           |
|Michigan      |6187           |
|New Jersey    |5218           |
|Massachusetts |4939           |
|Minnesota     |4897           |
|Maryland      |4827           |
+--------------+---------------+
only showing top 20 rows

In [72]:
covid = covid.select("submission_date", "state", "tot_cases")
covid.show(5)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+---------------+-----+---------+
|submission_date|state|tot_cases|
+---------------+-----+---------+
|           null|   KS|   297229|
|           null|   ND|   163565|
|           null|   AS|       11|
|           null|   AL|   841461|
|           null|   AK|   251425|
+---------------+-----+---------+
only showing top 5 rows

In [38]:
covid.dtypes

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

[('submission_date', 'string'), ('state', 'string'), ('tot_cases', 'int')]

In [67]:
df2 = covid.select(
    'submission_date', 'state', 'tot_cases',
    from_unixtime(unix_timestamp('submission_date', 'MM/dd/yyy')).alias('date')
)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [68]:
df2.show(5)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+---------------+-----+---------+-------------------+
|submission_date|state|tot_cases|               date|
+---------------+-----+---------+-------------------+
|     03/11/2021|   KS|   297229|2021-03-11 00:00:00|
|     12/01/2021|   ND|   163565|2021-12-01 00:00:00|
|     01/02/2022|   AS|       11|2022-01-02 00:00:00|
|     11/22/2021|   AL|   841461|2021-11-22 00:00:00|
|     05/30/2022|   AK|   251425|2022-05-30 00:00:00|
+---------------+-----+---------+-------------------+
only showing top 5 rows

In [ ]:
#Filtering only 2020 and 2021 year data for the pandemic

In [69]:
df2 = df2.withColumn('year',year(df2.date))
df2.show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+---------------+-----+---------+-------------------+----+
|submission_date|state|tot_cases|               date|year|
+---------------+-----+---------+-------------------+----+
|     03/11/2021|   KS|   297229|2021-03-11 00:00:00|2021|
|     12/01/2021|   ND|   163565|2021-12-01 00:00:00|2021|
|     01/02/2022|   AS|       11|2022-01-02 00:00:00|2022|
|     11/22/2021|   AL|   841461|2021-11-22 00:00:00|2021|
|     05/30/2022|   AK|   251425|2022-05-30 00:00:00|2022|
|     05/17/2020|  RMI|        0|2020-05-17 00:00:00|2020|
|     04/03/2020|   ND|      173|2020-04-03 00:00:00|2020|
|     09/04/2021|   PR|   173967|2021-09-04 00:00:00|2021|
|     05/09/2021|   PW|        0|2021-05-09 00:00:00|2021|
|     08/15/2022|   NM|   602931|2022-08-15 00:00:00|2022|
|     01/01/2022|   UT|   636992|2022-01-01 00:00:00|2022|
|     05/01/2021|   SD|   122688|2021-05-01 00:00:00|2021|
|     04/03/2021|   OH|  1024011|2021-04-03 00:00:00|2021|
|     10/16/2022|  NYC|  2908749|2022-10-16 00:00:00|202

In [60]:
print((df2.count(), len(df2.columns)))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

(60060, 5)

In [62]:
df2.dtypes

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

[('submission_date', 'string'), ('state', 'string'), ('tot_cases', 'int'), ('date', 'string'), ('year', 'int')]

In [70]:
li=[2020,2021]
data = df2.filter(df2.year.isin(li))
data.show(5)


VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+---------------+-----+---------+-------------------+----+
|submission_date|state|tot_cases|               date|year|
+---------------+-----+---------+-------------------+----+
|     03/11/2021|   KS|   297229|2021-03-11 00:00:00|2021|
|     12/01/2021|   ND|   163565|2021-12-01 00:00:00|2021|
|     11/22/2021|   AL|   841461|2021-11-22 00:00:00|2021|
|     05/17/2020|  RMI|        0|2020-05-17 00:00:00|2020|
|     04/03/2020|   ND|      173|2020-04-03 00:00:00|2020|
+---------------+-----+---------+-------------------+----+
only showing top 5 rows

### State-wise covid cases

In [74]:
f = data.groupBy("year","state").sum("tot_cases").withColumnRenamed("sum(tot_cases)", "total_cases") 
f.sort(f.total_cases.desc()).show(truncate=False)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+----+-----+-----------+
|year|state|total_cases|
+----+-----+-----------+
|2021|CA   |1472475446 |
|2021|TX   |1191617013 |
|2021|FL   |975849918  |
|2021|IL   |526709410  |
|2021|GA   |457894070  |
|2021|PA   |457469178  |
|2021|OH   |442495546  |
|2021|NY   |431719701  |
|2021|NC   |402933940  |
|2021|MI   |373501215  |
|2021|NJ   |370909011  |
|2021|TN   |354156998  |
|2021|AZ   |350452837  |
|2021|NYC  |345747620  |
|2021|IN   |298637568  |
|2021|WI   |267338308  |
|2021|VA   |263816216  |
|2021|MA   |263687704  |
|2021|MO   |250277309  |
|2021|SC   |244123204  |
+----+-----+-----------+
only showing top 20 rows

### Monthly Frequency of Flights Analysis 

In [38]:
monthly_19 = y19.groupBy('Month').count().withColumnRenamed("count", "2019_flights_frequency")

monthly_19 = monthly_19.sort("Month")
monthly_19.show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-----+----------------------+
|Month|2019_flights_frequency|
+-----+----------------------+
|    1|                638649|
|    2|                582966|
|    3|                689221|
|    4|                668259|
|    5|                694311|
|    6|                694469|
|    7|                717684|
|    8|                717456|
|    9|                660712|
|   10|                692944|
|   11|                655072|
|   12|                679941|
+-----+----------------------+

In [39]:
monthly_20 = y20.groupBy('Month').count().withColumnRenamed("count", "2020_flights_frequency")
monthly_20 = monthly_20.sort("Month")
monthly_20.show(5)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-----+----------------------+
|Month|2020_flights_frequency|
+-----+----------------------+
|    1|                660556|
|    2|                623103|
|    3|                701274|
|    4|                331238|
|    5|                192412|
+-----+----------------------+
only showing top 5 rows

In [40]:
monthly_21 = y21.groupBy('Month').count().withColumnRenamed("count", "2021_flights_frequency")
monthly_21 = monthly_21.sort("Month")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [41]:
#Joining all years data for flight frequency
all_months_data = monthly_19.join(monthly_20,monthly_19.Month ==  monthly_20.Month,"inner").drop(monthly_20.Month)
all_months_data = all_months_data.join(monthly_21,all_months_data.Month ==  monthly_21.Month,"inner").drop(monthly_21.Month)


VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [52]:
all_months_data.show(5)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-----+------+------+------+
|Month|  2019|  2020|  2021|
+-----+------+------+------+
|    1|638649|660556|379384|
|    2|582966|623103|350170|
|    3|689221|701274|467126|
|    4|668259|331238|473936|
|    5|694311|192412|520059|
+-----+------+------+------+
only showing top 5 rows

In [55]:
#Writing it to S3 to use it for reporting and visualization
all_months_data.repartition(1).write.csv("s3://airlinebdai/all_months_192021.csv")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [37]:
#Feature engineering for Delayed column

y19 = y19.withColumn('Delayed', when(col('DepDelay')>0, 1).otherwise(0))

y20 = y20.withColumn('Delayed', when(col('DepDelay')>0, 1).otherwise(0))

y21 = y21.withColumn('Delayed', when(col('DepDelay')>0, 1).otherwise(0))


VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

### Flight Cancellations, Delays before and during the pandemic

In [49]:
#Filtering and counting flight cancellations, delays, diversions
can_flights_19=y19.filter(y19.Cancelled == 1).groupBy('Month').count().withColumnRenamed("count", "Cancelled_count")
div_flights_19=y19.filter(y19.Diverted == 1).groupBy('Month').count().withColumnRenamed("count", "Diverted_count")
delay_flights_19=y19.filter(y19.Delayed == 1).groupBy('Month').count().withColumnRenamed("count", "Delayed_count")




VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [50]:
all_conditions_19 = monthly_19.join(can_flights_19, monthly_19.Month == can_flights_19.Month, "left").drop(monthly_19.Month)
all_conditions_19 = all_conditions_19.join(div_flights_19, all_conditions_19.Month == div_flights_19.Month,  "left").drop(div_flights_19.Month)
all_conditions_19 = all_conditions_19.join(delay_flights_19, all_conditions_19.Month == delay_flights_19.Month,  "left").drop(delay_flights_19.Month)
all_conditions_19.fillna(value = 0)
all_conditions_19.show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+----------------------+-----+---------------+--------------+-------------+
|2019_flights_frequency|Month|Cancelled_count|Diverted_count|Delayed_count|
+----------------------+-----+---------------+--------------+-------------+
|                679941|   12|           7176|          1549|       257291|
|                638649|    1|          19550|          1450|       201140|
|                694469|    6|          14667|          2778|       276949|
|                689221|    3|          13562|          1130|       223016|
|                668259|    4|          15726|          1616|       212523|
|                717456|    8|          12903|          1973|       251544|
|                655072|   11|           5561|          1043|       189232|
|                694311|    5|          14146|          2234|       241304|
|                582966|    2|          18352|          1787|       214337|
|                717684|    7|          14808|          2517|       257921|
|           

In [51]:
#For year 2020
can_flights_20=y20.filter(y20.Cancelled == 1).groupBy('Month').count().withColumnRenamed("count", "Cancelled_count")
div_flights_20=y20.filter(y20.Diverted == 1).groupBy('Month').count().withColumnRenamed("count", "Diverted_count")
delay_flights_20=y20.filter(y20.Delayed == 1).groupBy('Month').count().withColumnRenamed("count", "Delayed_count")



all_conditions_20 = monthly_20.join(can_flights_20, monthly_20.Month == can_flights_20.Month, "left").drop(monthly_20.Month)
all_conditions_20 = all_conditions_20.join(div_flights_20, all_conditions_20.Month == div_flights_20.Month,  "left").drop(div_flights_20.Month)
all_conditions_20 = all_conditions_20.join(delay_flights_20, all_conditions_20.Month == delay_flights_20.Month,  "left").drop(delay_flights_20.Month)
all_conditions_20.fillna(value = 0)
all_conditions_20.show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+----------------------+-----+---------------+--------------+-------------+
|2020_flights_frequency|Month|Cancelled_count|Diverted_count|Delayed_count|
+----------------------+-----+---------------+--------------+-------------+
|                397802|   12|           4253|           632|        84006|
|                660556|    1|           8416|          1290|       181463|
|                237264|    6|           1030|           373|        37491|
|                701274|    3|         118276|           874|       107348|
|                331238|    4|         136848|           315|        19009|
|                398470|    8|           4327|           858|        58365|
|                389587|   11|           2106|           591|        55431|
|                192412|    5|          12261|           373|        21333|
|                623103|    2|           6095|          1182|       172142|
|                370859|    7|           2926|           857|        57203|
|           

In [52]:
#For year 2021
can_flights_21=y21.filter(y21.Cancelled == 1).groupBy('Month').count().withColumnRenamed("count", "Cancelled_count")
div_flights_21=y21.filter(y21.Diverted == 1).groupBy('Month').count().withColumnRenamed("count", "Diverted_count")
delay_flights_21=y21.filter(y21.Delayed == 1).groupBy('Month').count().withColumnRenamed("count", "Delayed_count")



all_conditions_21 = monthly_21.join(can_flights_21, monthly_21.Month == can_flights_21.Month, "left").drop(monthly_21.Month)
all_conditions_21 = all_conditions_21.join(div_flights_21, all_conditions_21.Month == div_flights_21.Month,  "left").drop(div_flights_21.Month)
all_conditions_21 = all_conditions_21.join(delay_flights_21, all_conditions_21.Month == delay_flights_21.Month,  "left").drop(delay_flights_21.Month)
all_conditions_21.fillna(value = 0)
all_conditions_21.show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+----------------------+-----+---------------+--------------+-------------+
|2021_flights_frequency|Month|Cancelled_count|Diverted_count|Delayed_count|
+----------------------+-----+---------------+--------------+-------------+
|                580238|   12|          13773|          1453|       234952|
|                379384|    1|           4155|           620|        65834|
|                573779|    6|           9196|          1913|       237842|
|                467126|    3|           5904|           807|       103462|
|                473936|    4|           2561|           766|       111306|
|                611494|    8|          18734|          2174|       238882|
|                576693|   11|           3559|           765|       193797|
|                520059|    5|           2350|          1319|       150111|
|                350170|    2|          20201|           696|        78017|
|                615703|    7|          10195|          2359|       266386|
|           

In [53]:
#Exporting the three dataframes to S3 to further visualize locally 

all_conditions_19.repartition(1).write.csv("s3://airlinebdai/all_conditions_19.csv")
all_conditions_20.repartition(1).write.csv("s3://airlinebdai/all_conditions_20.csv")
all_conditions_21.repartition(1).write.csv("s3://airlinebdai/all_conditions_21.csv")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

### Analyzing covid and flights 

In [61]:
#joining 2021 and 2020 data with covid data
covid_all = covid.join(y20,covid.submission_date ==  y20.FlightDate,"inner").drop(y20.FlightDate)
covid_all = covid_all.join(y21,covid_all.submission_date ==  y21.FlightDate,"inner").drop(y21.FlightDate)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [80]:
#Function to convert to datetime
func =  udf (lambda x: datetime.strptime(x, '%m/%d/%Y'), DateType())

covid1 = covid.withColumn('submission_date', func(col('submission_date')))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [81]:
covid1.show(1)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+---------------+-----+---------+----------+----------+--------+---------+---------+----------+----------+---------+----------+--------------------+-------------+--------------+
|submission_date|state|tot_cases|conf_cases|prob_cases|new_case|pnew_case|tot_death|conf_death|prob_death|new_death|pnew_death|          created_at|consent_cases|consent_deaths|
+---------------+-----+---------+----------+----------+--------+---------+---------+----------+----------+---------+----------+--------------------+-------------+--------------+
|     2021-03-11|   KS|   297229|    241035|     56194|       0|        0|     4851|      null|      null|        0|         0|03/12/2021 03:20:...|        Agree|           N/A|
+---------------+-----+---------+----------+----------+--------+---------+---------+----------+----------+---------+----------+--------------------+-------------+--------------+
only showing top 1 row

In [25]:
#Function to convert to datetype 
func =  udf (lambda x: datetime.strptime(x, '%m/%d/%Y'), DateType())
covid = covid.select("submission_date", "tot_cases", "conf_cases")
covid = covid.withColumn("submission_date",func(col('submission_date')))

#Monthly covid cases
covid_1 = covid.groupBy('submission_date').sum()

covid_1 = covid_1.withColumn('year',year(covid_1.submission_date))
covid_1 = covid_1.withColumn('month',month(covid_1.submission_date))
covid_1.show(2)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+---------------+--------------+---------------+----+-----+
|submission_date|sum(tot_cases)|sum(conf_cases)|year|month|
+---------------+--------------+---------------+----+-----+
|     2022-08-02|      91554444|       50898563|2022|    8|
|     2022-05-26|      83766799|       46812055|2022|    5|
+---------------+--------------+---------------+----+-----+
only showing top 2 rows

In [33]:
total_cases_21 = covid_1.filter(covid_1.year == 2021)
total_cases_21 = total_cases_21.groupby('month').mean('sum(tot_cases)').withColumnRenamed('avg(sum(tot_cases))', 'cases_2021')


total_cases_20 = covid_1.filter(covid_1.year == 2020)
total_cases_20 = total_cases_20.groupby('month').mean('sum(tot_cases)').withColumnRenamed('avg(sum(tot_cases))', 'cases_2020')



VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

#### Monthly cases for 2020 and 2021

In [36]:

all_mnt_covid = total_cases_21.join(total_cases_20,total_cases_21.month ==  total_cases_20.month,"left").drop(total_cases_20.month)
all_mnt_covid=all_mnt_covid.fillna(value = 0).sort('month')
all_mnt_covid.show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------------+-----+-------------------+
|          cases_2021|month|         cases_2020|
+--------------------+-----+-------------------+
| 2.358491351612903E7|    1|                7.3|
| 2.759076264285714E7|    2|  33.51724137931034|
|2.9468835258064516E7|    3| 44893.354838709674|
|3.1358997266666666E7|    4|  672149.1333333333|
|3.2736336870967742E7|    5| 1488751.2258064516|
|3.3303102866666667E7|    6| 2208468.7333333334|
| 3.409814729032258E7|    7|  3666248.709677419|
| 3.706526980645161E7|    8|  5437575.903225807|
| 4.159162013333333E7|    9|  6729035.433333334|
|4.4809639258064516E7|   10|  8203875.258064516|
|4.7237491166666664E7|   11|       1.14503566E7|
| 5.088725632258064E7|   12|1.709293793548387E7|
+--------------------+-----+-------------------+

### Airline Company Analysis

In [55]:
#Function for analysing airline wise delay, on time, cancellation counts
def get_delay_descstates_df(df):
    df_counts = df.groupby('Airline').agg(TotalFlightCounts = ('DepDelay', lambda x: x.count()),
                                                     DelayDepCounts = ('DepDelay', lambda x: (x > 0).sum()),
                                                     EarlyDepCounts = ('DepDelay', lambda x: (x < 0).sum()),
                                                     OntimeDepCounts = ('DepDelay', lambda x: (x == 0).sum()),
                                                     DelayArrCounts = ('ArrDelay', lambda x: (x > 0).sum()),
                                                     EarlyArrCounts = ('ArrDelay', lambda x: (x < 0).sum()),
                                                     OntimeArrCounts = ('ArrDelay', lambda x: (x == 0).sum()))
    
    df_counts['DelayDepRatio'] = df_counts['DelayDepCounts'] / df_counts['TotalFlightCounts']
    df_counts['EarlyDepRatio'] = df_counts['EarlyDepCounts'] / df_counts['TotalFlightCounts']
    df_counts['OntimeDepRatio'] = df_counts['OntimeDepCounts'] / df_counts['TotalFlightCounts']
    df_counts['DelayArrRatio'] = df_counts['DelayArrCounts'] / df_counts['TotalFlightCounts']
    df_counts['EarlyArrRatio'] = df_counts['EarlyArrCounts'] / df_counts['TotalFlightCounts']
    df_counts['OntimeArrRatio'] = df_counts['OntimeArrCounts'] / df_counts['TotalFlightCounts']
    return df_counts
delay = udf(lambda m: get_delay_descstates_df(m), StructType())
spark.udf.register("delay", delay)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

<function <lambda> at 0x7f3a2277c170>

In [58]:
#Running the above function
cnt_cond = lambda cond: sum(when(cond, 1).otherwise(0))
delaydepcounts =  merged.groupBy('Airline').agg(cnt_cond(col('DepDelay') > 0).alias('Number of Delayed Departures'))
earlyontimedepcounts =  merged.groupBy('Airline').agg(cnt_cond(col('DepDelay') <= 0).alias('EarlyOntimeDepCounts'))
delayarrcounts =  merged.groupBy('Airline').agg(cnt_cond(col('ArrDelay') > 0).alias('DelayArrCounts'))
earlyontimearrcounts =  merged.groupBy('Airline').agg(cnt_cond(col('ArrDelay') <= 0).alias('EarlyOntimeArrCounts'))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [109]:
delaydepcounts.show(truncate= False)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-----------------------------------------+--------------+
|Airline                                  |DelayDepCounts|
+-----------------------------------------+--------------+
|United Air Lines Inc.                    |419694        |
|Compass Airlines                         |28316         |
|Southwest Airlines Co.                   |1379909       |
|Commutair Aka Champlain Enterprises, Inc.|51319         |
|Alaska Airlines Inc.                     |170309        |
|Allegiant Air                            |101745        |
|SkyWest Airlines Inc.                    |506783        |
|Horizon Air                              |84299         |
|Air Wisconsin Airlines Corp              |53441         |
|Trans States Airlines                    |27774         |
|ExpressJet Airlines Inc.                 |46096         |
|Capital Cargo International              |58755         |
|Hawaiian Airlines Inc.                   |50644         |
|GoJet Airlines, LLC d/b/a United Express |43440        

In [59]:
merged_t =  merged.groupBy('Airline').agg(count('DepDelay').alias('TotalFlightCounts'),
                                    cnt_cond(col('DepDelay') > 0).alias('DelayedDepartureFlightCounts'),
                                    cnt_cond(col('DepDelay') <= 0).alias('EarlyOntimeDepartureFlightCounts'),
                                    cnt_cond(col('ArrDelay') > 0).alias('DelayedArrivalFlightCounts'),
                                    cnt_cond(col('ArrDelay') <= 0).alias('EarlyOntimeArrivalFlightCounts'))

merged_t = merged_t.withColumn("Departure Delayed Ratio", merged_t.DelayedDepartureFlightCounts / merged_t.TotalFlightCounts)
merged_t = merged_t.withColumn("Departure Early/Ontime Ratio", merged_t.EarlyOntimeDepartureFlightCounts / merged_t.TotalFlightCounts)
merged_t = merged_t.withColumn("Arrival Delayed Ratio", merged_t.DelayedArrivalFlightCounts / merged_t.TotalFlightCounts)
merged_t = merged_t.withColumn("Arrival Early/Ontime Ratio", merged_t.EarlyOntimeArrivalFlightCounts / merged_t.TotalFlightCounts)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [60]:
airlines = merged_t.select("Airline", "Departure Delayed Ratio", "Departure Early/Ontime Ratio" , "Arrival Delayed Ratio", "Arrival Early/Ontime Ratio")
airlines.show(5)


VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------------+-----------------------+----------------------------+---------------------+--------------------------+
|             Airline|Departure Delayed Ratio|Departure Early/Ontime Ratio|Arrival Delayed Ratio|Arrival Early/Ontime Ratio|
+--------------------+-----------------------+----------------------------+---------------------+--------------------------+
|United Air Lines ...|    0.31150167925333533|          0.6884983207466647|  0.32018703727756853|        0.6769918171191064|
|    Compass Airlines|     0.2615410193412521|          0.7384589806587479|   0.3232224336356751|        0.6751242310605361|
|Southwest Airline...|     0.4238721492718133|          0.5761278507281866|  0.31317125195516243|        0.6847559379803433|
|Commutair Aka Cha...|     0.3192790573245238|          0.6807209426754762|   0.4154441499620491|        0.5800204063857056|
|Alaska Airlines Inc.|     0.2933044694276818|          0.7066955305723182|  0.33713076244798984|        0.6596504643024441|


In [61]:
#Exporting to S3 to use it for visualizations
airlines.repartition(1).write.csv("s3://airlinebdai/airlines.csv")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [82]:
cnt_cond = lambda cond: sum(when(cond, 1).otherwise(0))
merged.groupBy('Airline').agg(cnt_cond(col('DepDelay') > 0)).withColumnRenamed("sum(CASE WHEN (DepDelay > 0) THEN 1 ELSE 0 END)", "Count of Delayed Flights").show(truncate=False)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-----------------------------------------+------------------------+
|Airline                                  |Count of Delayed Flights|
+-----------------------------------------+------------------------+
|United Air Lines Inc.                    |419694                  |
|Compass Airlines                         |28316                   |
|Southwest Airlines Co.                   |1379909                 |
|Commutair Aka Champlain Enterprises, Inc.|51319                   |
|Alaska Airlines Inc.                     |170309                  |
|Allegiant Air                            |101745                  |
|SkyWest Airlines Inc.                    |506783                  |
|Horizon Air                              |84299                   |
|Air Wisconsin Airlines Corp              |53441                   |
|Trans States Airlines                    |27774                   |
|ExpressJet Airlines Inc.                 |46096                   |
|Capital Cargo International      

### Predictive Analytics 

In [7]:
#Predicting if a given flight will be cancelled 
from pyspark.sql.functions import round
from pyspark.ml.feature import StringIndexer
from pyspark.ml.feature import OneHotEncoderEstimator, VectorAssembler
from pyspark.ml.classification import DecisionTreeClassifier
from pyspark.ml.classification import LogisticRegression
from pyspark.ml.evaluation import MulticlassClassificationEvaluator, BinaryClassificationEvaluator
from pyspark.ml import Pipeline

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [8]:
#Exploring data types of columns for feature engineering
merged.dtypes

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

[('FlightDate', 'timestamp'), ('Airline', 'string'), ('Origin', 'string'), ('Dest', 'string'), ('Cancelled', 'int'), ('Diverted', 'int'), ('CRSDepTime', 'int'), ('DepTime', 'double'), ('DepDelayMinutes', 'double'), ('DepDelay', 'double'), ('ArrTime', 'double'), ('ArrDelayMinutes', 'double'), ('AirTime', 'double'), ('CRSElapsedTime', 'double'), ('ActualElapsedTime', 'double'), ('Distance', 'double'), ('Year', 'int'), ('Quarter', 'int'), ('Month', 'int'), ('DayofMonth', 'int'), ('DayOfWeek', 'int'), ('IATA_Code_Marketing_Airline', 'string'), ('Flight_Number_Marketing_Airline', 'int'), ('Operating_Airline', 'string'), ('Flight_Number_Operating_Airline', 'int'), ('OriginAirportID', 'int'), ('OriginAirportSeqID', 'int'), ('OriginCityName', 'string'), ('OriginState', 'string'), ('OriginStateName', 'string'), ('DestAirportID', 'int'), ('DestAirportSeqID', 'int'), ('DestCityMarketID', 'int'), ('DestCityName', 'string'), ('DestState', 'string'), ('DestStateName', 'string'), ('DepDel15', 'double

In [9]:
#Selecting features for our model
merged = merged.select("Airline", "Origin", "Dest", "Cancelled", "Diverted", "DepDelay", "ArrDelay", "AirTime", "Year", "Month", "DayofMonth", "Quarter", "OriginStateName", "DestStateName", "Distance")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [10]:
df_Columns = ["Airline", "Origin", "Dest", "Cancelled", "Diverted", "DepDelay", "ArrDelay", "AirTime", "Year", "Month", "DayofMonth", "Quarter", "OriginStateName", "DestStateName", "Distance"]
merged.select([count(when(isnan(c) | col(c).isNull(), c)).alias(c) for c in df_Columns]).show()



VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-------+------+----+---------+--------+--------+--------+-------+----+-----+----------+-------+---------------+-------------+--------+
|Airline|Origin|Dest|Cancelled|Diverted|DepDelay|ArrDelay|AirTime|Year|Month|DayofMonth|Quarter|OriginStateName|DestStateName|Distance|
+-------+------+----+---------+--------+--------+--------+-------+----+-----+----------+-------+---------------+-------------+--------+
|      0|     0|   0|        0|       0|  556014|  609888| 609888|   0|    0|         0|      0|              0|            0|       0|
+-------+------+----+---------+--------+--------+--------+-------+----+-----+----------+-------+---------------+-------------+--------+

In [11]:
#We have to fill nas with 0 for rows which have cancelled as true
merged = merged.fillna(value=0)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [12]:
#Storing column names in a variable to use it in our pipeline stages
cols = merged.columns
cols

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

['Airline', 'Origin', 'Dest', 'Cancelled', 'Diverted', 'DepDelay', 'ArrDelay', 'AirTime', 'Year', 'Month', 'DayofMonth', 'Quarter', 'OriginStateName', 'DestStateName', 'Distance']

In [13]:
#Storing categorical columns in a seperate variable
categoricalColumns = ["Airline", "Origin", "Dest", "OriginStateName", "DestStateName"]

#Initiating stages of a pipeline
stages = []

#Converting categorical to numerical using a string indexer
for categoricalCol in categoricalColumns:
    stringIndexer = StringIndexer(inputCol = categoricalCol, outputCol = categoricalCol + 'Index')
    encoder = OneHotEncoderEstimator(inputCols=[stringIndexer.getOutputCol()], outputCols=[categoricalCol + "classVec"])
    stages += [stringIndexer, encoder]
    
    
#Doing the same for the target variable     
label_stringIdx = StringIndexer(inputCol = 'Cancelled', outputCol = 'label')
stages += [label_stringIdx]

numericCols = ["Diverted", "DepDelay", "ArrDelay", "AirTime", "Year", "Month", "DayofMonth", "Quarter", "Distance"]

#Vector Assembler to convert  input columns into a vector
assemblerInputs = [c + "classVec" for c in categoricalColumns] + numericCols
assembler = VectorAssembler(inputCols=assemblerInputs, outputCol="features")
stages += [assembler]

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [14]:
#Creating a pipeline
pipeline = Pipeline(stages = stages)
pipelineModel = pipeline.fit(merged)
merged = pipelineModel.transform(merged)
selectedCols = ['label', 'features'] + cols
merged = merged.select(selectedCols)
merged.printSchema()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- label: double (nullable = false)
 |-- features: vector (nullable = true)
 |-- Airline: string (nullable = true)
 |-- Origin: string (nullable = true)
 |-- Dest: string (nullable = true)
 |-- Cancelled: integer (nullable = false)
 |-- Diverted: integer (nullable = false)
 |-- DepDelay: double (nullable = false)
 |-- ArrDelay: double (nullable = false)
 |-- AirTime: double (nullable = false)
 |-- Year: integer (nullable = true)
 |-- Month: integer (nullable = true)
 |-- DayofMonth: integer (nullable = true)
 |-- Quarter: integer (nullable = true)
 |-- OriginStateName: string (nullable = true)
 |-- DestStateName: string (nullable = true)
 |-- Distance: double (nullable = false)

In [15]:
#Splitting data into train and test
train, test = merged.randomSplit([0.7, 0.3], seed = 42)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

#### Logistic Regression Model

In [16]:
#Fitting and transforming
lr = LogisticRegression(featuresCol = 'features', labelCol = 'label', maxIter=10)
lrModel = lr.fit(train)
predictions = lrModel.transform(test)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [19]:
#Evaluating the model using the metric- Accuracy
evaluator=MulticlassClassificationEvaluator(labelCol = "label", metricName="accuracy")
print('Accuracy', evaluator.evaluate(predictions))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Accuracy 0.9754975980628011

In [26]:
#Examining the model summary
trainingSummary_lr = lrModel.summary
trainingSummary_lr.roc.show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------------+-------------------+
|                 FPR|                TPR|
+--------------------+-------------------+
|                 0.0|                0.0|
|0.003988670198095797|0.28931207804031644|
|0.010790475094652991| 0.4405883287559298|
|0.011286701208326873|0.44849979698017467|
|0.019356984581388847|  0.543813191496874|
| 0.02677060726456548| 0.5985503627876009|
| 0.03581713276142977|   0.64791570515973|
|0.045070526448867006| 0.6887819567075985|
|0.047720045148925784| 0.6988421564992825|
| 0.05716182447896826| 0.7321954135173374|
| 0.06640059877395564| 0.7616017633721971|
|  0.0760170533319224| 0.7887786781265683|
| 0.08343105475583608| 0.8065258894412037|
| 0.09318324999395909| 0.8269136194777977|
| 0.10300240659439132| 0.8436570806001317|
| 0.10626101614776282|  0.848355539414849|
| 0.11615675412522516| 0.8615757869224968|
| 0.12608210962832428|   0.87339633252882|
| 0.12917755767217762| 0.8768211887126021|
| 0.13913866630085361| 0.8872647302862705|
+----------

#### Random Forest

In [20]:
from pyspark.ml.classification import RandomForestClassifier
rf = RandomForestClassifier(featuresCol = 'features', labelCol = 'label')
rfModel = rf.fit(train)
predictions_rf = rfModel.transform(test)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [27]:
evaluator_rf=MulticlassClassificationEvaluator(labelCol = "label", metricName="accuracy")
print('Accuracy', evaluator_rf.evaluate(predictions_rf))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Accuracy 0.9709686131772629

---